# 电影数据分析
## 定义公用函数

In [7]:
import pandas as pd
import numpy as np
import types

#根据|解析字段值
def split_column(x):
    res = []
    res = x.split('|', 5)
    for i in range(5 - len(res)):
        res.append('')
    return res

#将|分割的数据解析，然后转化成每行一个 
def convert_columns(df, column_name):
    value_vars = []
    for i in range(5):
        value_vars.append("%s%d" % (column_name, i))
    
    var_name = column_name + '_index'
    value_name = column_name + '_adj'
    
    columns = df.columns.values 
    #将单列数据分割成多列
    df[value_vars[0]], df[value_vars[1]], df[value_vars[2]], df[value_vars[3]], df[value_vars[4]] = \
    zip(*df[column_name].apply(split_column))

    #将多列数据转化成多行数据
    df = pd.melt(df, id_vars = columns, value_vars = value_vars, var_name = var_name, value_name = value_name)
    
    #删除转化生成的空白数据
    df = df[df[value_name] != '']
    #删除多余字段
    del df[var_name]
    del df[column_name]
    df.rename(columns={value_name:column_name}, inplace = True)
    
    return df

## 载入数据

In [8]:
movies_df = pd.read_csv('movies.csv')
print len(movies_df)
movies_df.head()

10866


,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-04-01,2947,7.3,2015,1.747999e+08,1.385749e+09


## 熟悉数据
### 信息的格式
  * id：标识号
  * imdb_id：IMDB 标识号
  * popularity：在 Movie Database 上的相对页面查看次数
  * budget：预算（美元）
  * revenue：收入（美元）
  * original_title：电影名称
  * cast：演员列表，按 | 分隔，最多 5 名演员
  * homepage：电影首页的 URL
  * director：导演列表，按 | 分隔，最多 5 名导演
  * tagline：电影的标语
  * keywords：与电影相关的关键字，按 | 分隔，最多 5 个关键字
  * overview：剧情摘要
  * runtime：电影时长
  * genres：风格列表，按 | 分隔，最多 5 种风格
  * production_companies：制作公司列表，按 | 分隔，最多 5 家公司
  * release_date：首次上映日期
  * vote_count：评分次数
  * vote_average：平均评分
  * release_year：发行年份
  * budget_adj：根据通货膨胀调整的预算（2010 年，美元）
  * revenue_adj：根据通货膨胀调整的收入（2010 年，美元）

In [9]:
#查看哪些字段存在数据缺失的情况
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
id                      10866 non-null int64
imdb_id                 10856 non-null object
popularity              10866 non-null float64
budget                  10866 non-null int64
revenue                 10866 non-null int64
original_title          10866 non-null object
cast                    10790 non-null object
homepage                2936 non-null object
director                10822 non-null object
tagline                 8043 non-null object
keywords                9373 non-null object
overview                10862 non-null object
runtime                 10866 non-null int64
genres                  10843 non-null object
production_companies    9836 non-null object
release_date            10866 non-null object
vote_count              10866 non-null int64
vote_average            10866 non-null float64
release_year            10866 non-null int64
budget_adj              1

从上面的信息中可以看到，IMDB 标识号、制作公司列表、风格列表信息不全。
## 数据清理

In [10]:
#清除imdb_id为空的行
movies_df = movies_df[movies_df['imdb_id'].notnull()]

#清除genres为空的行
movies_df = movies_df[movies_df['genres'].notnull()]

#解析genres字段，转化成Tableau适合处理的格式
movies_df = convert_columns(movies_df, 'genres')
movies_df.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj,genres
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,5562,6.5,2015,1.379999e+08,1.392446e+09,Action
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,6185,7.1,2015,1.379999e+08,3.481613e+08,Action
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,2480,6.3,2015,1.012000e+08,2.716190e+08,Adventure
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,5292,7.5,2015,1.839999e+08,1.902723e+09,Action
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Universal Pictures|Original Film|Media Rights ...,2015-04-01,2947,7.3,2015,1.747999e+08,1.385749e+09,Action


In [11]:
#清除production_companies为空的行
movies_df = movies_df[movies_df['production_companies'].notnull()]

#解析production_companies字段，转化成Tableau适合处理的格式
movies_df = convert_columns(movies_df, 'production_companies')
movies_df.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj,genres,production_companies
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,2015-06-09,5562,6.5,2015,1.379999e+08,1.392446e+09,Action,Universal Studios
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,2015-05-13,6185,7.1,2015,1.379999e+08,3.481613e+08,Action,Village Roadshow Pictures
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,2015-03-18,2480,6.3,2015,1.012000e+08,2.716190e+08,Adventure,Summit Entertainment
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,2015-12-15,5292,7.5,2015,1.839999e+08,1.902723e+09,Action,Lucasfilm
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,2015-04-01,2947,7.3,2015,1.747999e+08,1.385749e+09,Action,Universal Pictures


In [12]:
#清除keywords为空的行
movies_df = movies_df[movies_df['keywords'].notnull()]

#添加字段表示是否原创
movies_df['is_original'] = movies_df['keywords'].apply(lambda x: 'based on novel' not in x)
movies_df.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,runtime,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj,genres,production_companies,is_original
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,124,2015-06-09,5562,6.5,2015,1.379999e+08,1.392446e+09,Action,Universal Studios,True
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,120,2015-05-13,6185,7.1,2015,1.379999e+08,3.481613e+08,Action,Village Roadshow Pictures,True
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,119,2015-03-18,2480,6.3,2015,1.012000e+08,2.716190e+08,Adventure,Summit Entertainment,False
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,136,2015-12-15,5292,7.5,2015,1.839999e+08,1.902723e+09,Action,Lucasfilm,True
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,137,2015-04-01,2947,7.3,2015,1.747999e+08,1.385749e+09,Action,Universal Pictures,True


In [13]:
#保存到csv文件，Tableau进行处理
movies_df.to_csv('movies-adj.csv', index = False)